In [7]:
!pip install pyocr
!pip install pytesseract
!apt install tesseract-ocr
!apt search tesseract-ocr-jpn

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Sorting... Done
Full Text Search... Done
tesseract-ocr-jpn/jammy,now 1:4.00~git30-7274cfa-1.1 all [installed]
  tesseract-ocr language files for Japanese

tesseract-ocr-jpn-vert/jammy,now 1:4.00~git30-7274cfa-1.1 all [installed]
  tesseract-ocr language files for Japanese (vertical)



In [8]:
!apt-get update
!apt install -y tesseract-ocr-jpn
!apt install -y tesseract-ocr-jpn-vert

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-jpn is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded

In [25]:
import pyocr
import pytesseract
from PIL import Image

tools = pyocr.get_available_tools()
tool = tools[0]

# img = Image.open('/content/drive/MyDrive/1月分.PNG')
# img = Image.open('/content/drive/MyDrive/12月分.PNG')
img = Image.open('/content/drive/MyDrive/2024_3月分.PNG')



text = tool.image_to_string(
    img,
    lang="jpn",
    builder=pyocr.builders.TextBuilder(tesseract_layout=6)
)

print(text)



yiリ::還還のmAWEC1上り9にに9に4上6NE:|E
0に7パ
\ 3.056
メモを入力 (最大60文字)
に|                       2024/02/22
支払い方法                                                1回払い
利用者                            と


In [26]:
import re
import openpyxl
import os
from datetime import datetime

amountPattern = r'\\?(\d+\.\d{3})'
dateOfUsePattern = r'\d{4}/\d{2}/\d{2}'

# ファイルが存在するか確認
file_path = '/content/drive/MyDrive/2024年-月分_立替経費精算所　(清水凪).xlsx'
if os.path.exists(file_path):
    existing_workbook = openpyxl.load_workbook(file_path)
else:
    # 既存のファイルが存在しない場合、新しいワークブックを作成
    existing_workbook = openpyxl.Workbook()

# シートを取得
sheet = existing_workbook.active

# 一番番号が大きいセルを探す
max_row = sheet.max_row if sheet.max_row else 0

# マッチするすべてのパターンを抽出
amountMatches = re.findall(amountPattern, text)
dateOfUseMatch = re.search(dateOfUsePattern, text)

# Checking and printing the extracted date
if dateOfUseMatch and len(set(amountMatches)) == 1:
    date_of_use = dateOfUseMatch.group(0)
    sheet.cell(row=13, column=1, value=date_of_use)
    print(f"Date of use {date_of_use} added to cell A13")
    # existing_workbook.save(file_path)

    result = amountMatches[0].replace('.', '')  # 小数点を削除
    result_numeric = float(result)
    #write amount of cost result data.
    sheet.cell(row=13, column=4, value=result_numeric)
    print(f"Value {result} added to cell A{max_row + 1}")

    today_date = datetime.now().strftime('%Y/%m/%d')

    # F3セルに今日の日付を書き込む
    sheet['F3'] = today_date
    print(f"Today's date {today_date} added to cell F3")

    # 利用日から年と月を抽出
    year, month = re.match(r'(\d{4})/(\d{2})/\d{2}', date_of_use).groups()

    # 新しいファイル名を生成
    new_file_name = f'/content/drive/MyDrive/{year}年{int(month)}月分_立替経費精算所　(清水凪).xlsx'

    # Excelファイルを新しい名前で保存
    existing_workbook.save(new_file_name)
    print(f"File saved as {new_file_name}")
else:
    print("No date or amount found.")


Date of use 2024/02/22 added to cell A13
Value 3056 added to cell A1001
Today's date 2024/03/03 added to cell F3
File saved as /content/drive/MyDrive/2024年2月分_立替経費精算所　(清水凪).xlsx


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
